In [1]:
suppressMessages(library("rwwa"))

# DSR attribution

In [2]:
nsamp = 1000
rp <- 35
varnm <- "dsr-june"

## CORDEX

In [3]:
fl <- list.files("ts_cordex", pattern = varnm, full.names = T)
for (fnm in fl) {
    
    mnm <- paste0(gsub(".dat","",strsplit(fnm, "_")[[1]][4:7]), collapse = "_")
    gmst_fnm <- list.files("ts_cordex", pattern = paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][5],"_rcp85_",substr(strsplit(fnm, "_")[[1]][6],1,2)), full.names = T)
    if(length(gmst_fnm) == 0) { cat("GMST missing:", mnm); next }
    
    # load data & combine all covariates with time series
    df <- merge(load_ts(fnm, col.names = c("year", "dsr")), load_ts(gmst_fnm, col.names = c("year", "gmst")))
    df$log_dsr <- log(df$dsr)
    mdl <- fit_ns("norm", "shift", df, "log_dsr", "gmst", lower = F)
    res_fnm <- paste0("res/res-cordex_logdsr_",mnm,".csv")
    
    if(!file.exists(res_fnm)) {
        
        # use model's 2023 GMST & observed Nino to define factual climate        
        cov_now <- df[df$year == 2024,"gmst",drop = F]
        cov_hist <- cov_now - 1.2
        cov_fut <- cov_now + 0.8

        # bootstrap results
        res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                             y_now = 2024, y_start = 1979, y_fut = 2050, nsamp = nsamp, di_relative = T) # relative change in intensity is already adjusted back to original variable
        write.csv(res, res_fnm)
    }
    
    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res/", "mdl-eval/rlplot_", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = mnm)
        }; dev.off()
    }
}

In [6]:
# compile results
res <- t(sapply(list.files("res", pattern = "cordex_logdsr", full.names = T), read.csv, row.names = "X"))
rownames(res) <- sapply(rownames(res), function(r) gsub(".csv","",paste0(strsplit(r, "_")[[1]][c(3,4,6)], collapse = "_")))
write.csv(res, "res-cordex_dsr-june.csv")

## CMIP6

In [ ]:
fl <- list.files("ts_cmip6", pattern = varnm, full.names = T)
for (fnm in fl[1]) {
    
    mnm <- gsub(".dat","",strsplit(fnm, "_")[[1]][5])
    gmst_fnm <- list.files("ts_cmip6", pattern = paste0("SGSAT2023.", mnm), full.names = T)
    if(length(gmst_fnm) == 0) { cat("GMST missing:", mnm,"\n"); next }
    
    # load data & combine all covariates with time series
    df <- merge(load_ts(fnm, col.names = c("year", "dsr")), load_ts(gmst_fnm, col.names = c("year", "gmst")))
    df$log_dsr <- log(df$dsr)
    mdl <- fit_ns("norm", "shift", df, "log_dsr", "gmst", lower = F)
    res_fnm <- paste0("res/res-cmip6_logdsr_",mnm,".csv")
    
    if(!file.exists(res_fnm)) {
        
        # use model's 2023 GMST & observed Nino to define factual climate        
        cov_now <- df[df$year == 2024,"gmst",drop = F]
        cov_hist <- cov_now - 1.2
        cov_fut <- cov_now + 0.8

        # bootstrap results
        res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                             y_now = 2024, y_start = 1979, y_fut = 2050, nsamp = nsamp, di_relative = T) # relative change in intensity is already adjusted back to original variable
        write.csv(res, res_fnm)
    }
    
    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res/", "mdl-eval/rlplot_", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = mnm)
        }; dev.off()
    }
}

In [8]:
# compile results
res <- t(sapply(list.files("res", pattern = "cmip6_logdsr", full.names = T), read.csv, row.names = "X"))
rownames(res) <- sapply(rownames(res), function(r) gsub(".csv","",paste0(strsplit(r, "_")[[1]][3], collapse = "_")))
write.csv(res, "res-cmip6_dsr-june.csv")

---
# JJ-precip attribution

In [2]:
nsamp = 1000
rp <- 100

## CORDEX

In [ ]:
fl <- list.files("ts_cordex", pattern = "pr-jj", full.names = T)
for (fnm in fl) {
    
    mnm <- paste0(gsub(".dat","",strsplit(fnm, "_")[[1]][4:7]), collapse = "_")
    gmst_fnm <- list.files("ts_cordex", pattern = paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][5],"_rcp85_",substr(strsplit(fnm, "_")[[1]][6],1,2)), full.names = T)
    if(length(gmst_fnm) == 0) { cat("GMST missing:", mnm); next }
    
    # load data & combine all covariates with time series
    df <- merge(load_ts(fnm, col.names = c("year", "pr")), load_ts(gmst_fnm, col.names = c("year", "gmst")))
    mdl <- fit_ns("norm", "fixeddisp", df, "pr", "gmst", lower = F)
    res_fnm <- paste0("res/res-cordex_pr-jj_",mnm,".csv")
    
    if(!file.exists(res_fnm)) {
        
        # use model's 2023 GMST & observed Nino to define factual climate        
        cov_now <- df[df$year == 2024,"gmst",drop = F]
        cov_hist <- cov_now - 1.2
        cov_fut <- cov_now + 0.8

        # bootstrap results
        res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                             y_now = 2024, y_start = 1979, y_fut = 2050, nsamp = nsamp)
        write.csv(res, res_fnm)
    }
    
    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res/", "mdl-eval/rlplot_", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = mnm)
        }; dev.off()
    }
}

In [104]:
# compile results
res <- t(sapply(list.files("res", pattern = "cordex_pr-jj", full.names = T), read.csv, row.names = "X"))
rownames(res) <- sapply(rownames(res), function(r) gsub(".csv","",paste0(strsplit(r, "_")[[1]][c(3,4,6)], collapse = "_")))
write.csv(res, "res-cordex_pr-jj.csv")